# Report Notebook

### GCN-LPA Report
The GCN-LPA approach aims to unify node feature propagation along with edge propagation. This means that we can learn the weights of the edges of a network by assessing how a feature of a node relates to the features of its neighbors and how this feature influences those features that are on the other side of the edges that connect to that node.

So rather than looking at different layers of neighbors like in graphSage, here we analyze the importance of a neighbor through edge weights. The goal is to have edge weights that are learnable so that we can improve our classification accuracy. 

The LPA propagates labels of known nodes to those nodes that have not been labeled and through this finds communities within a network. Once we minimize the loss of the predictions reached through the LPA we reach and learn the optimal edge weights of the network. Since now we believe to have the optimal edge weights we feed them into a GCN which will learn and classify node labels. 

Here we can see how the LPA helps the GCN by acting as a regularizer which leads to better results when compared to just a GCN.  This further shows how the GCN-LPA approach differs from the graphsage approach, rather than using aggregated information through a fully connected network we use an optimized graph with optimized edge weights into a fully connected network that will then learn over our features.

### GCN-LPA Model Code

In [11]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.optim as optim

In [12]:
class GCNLPA(nn.Module):
    def __init__(self, nfeat, nhid, nclass, adj, dropout_rate):
        super(GCNLPA, self).__init__()

        self.gc1 = GCNLPAConv(nfeat, nhid, adj)
        self.gc2 = GCNLPAConv(nhid, nclass, adj)
        self.dropout = dropout_rate

    def forward(self, x, adj, y):
        x, y_hat = self.gc1(x, adj, y)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x, y_hat = self.gc2(x, adj, y_hat)
        return F.log_softmax(x, dim=1), F.log_softmax(y_hat,dim=1)

In [13]:
class GCNLPAConv(nn.Module):
    """
    A GCN-LPA layer. Please refer to: https://arxiv.org/abs/2002.06755
    """

    def __init__(self, in_features, out_features, adj, bias=True):
        super(GCNLPAConv, self).__init__()
#         Creating features from given data
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
#         Reset for LPA so unlabeled nodes do not overpowered labeled
        self.reset_parameters()
        self.adjacency_mask = Parameter(adj.clone()).to_dense()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj, y):
        #Creating adjacency matrix
        adj = adj.to_dense()
        # W * x
        support = torch.mm(x, self.weight)
        
        # Row-Normalize: D^-1 * (A') to normalize edge weights
        adj = adj * self.adjacency_mask
        adj = F.normalize(adj, p=1, dim=1) 

        # output = D^-1 * A' * X * W
        output = torch.mm(adj, support)
        # y' = D^-1 * A' * y ; matrix multiplcation to propagate labels to neighbors
        y_hat = torch.mm(adj, y)

        if self.bias is not None:
            return output + self.bias, y_hat
        else:
            return output, y_hat

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'